In [1]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

In [2]:
#Global variables

background = None

#ROI

roi_top = 50
roi_bottom = 300
roi_right = 250
roi_left = 600


#Alpha for accumulating images

alpha = 0.5

In [3]:
#average background value

def running_bg(frame) :
    global background, alpha
    if background is None:
        background = frame.copy().astype('float')
        
    cv2.accumulateWeighted(frame, background, alpha)

In [4]:
#segmenting the hand
def segmenting(frame) :
    diff = cv2.absdiff(background.astype('uint8'), frame)
    _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
    image, contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None
    else:
        hand = max(contours, key = cv2.contourArea)
        return (hand, thresh)

In [5]:
#Counting the fingers

def fingers(hand, thresh):
    conv_hull = cv2.convexHull(hand)
    
    count = 0
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    #center
    cx = (left[0] + right[0]) //2
    cy = (top[1] + bottom[1]) //2
    
    distance = pairwise.euclidean_distances([(cx,cy)], Y = [left, right, top, bottom])[0]
    max_d = distance.max()
    
    radius = int(0.8*max_d)
    circumference = 2*np.pi*radius
    
    circular = np.zeros(thresh.shape[:2], dtype="uint8")
    
    cv2.circle(circular, (cx,cy), radius, 255, 10)
    cv2.imshow('frame1',circular)
    circular = cv2.bitwise_and(thresh, thresh, mask = circular)
    cv2.imshow('frame2',circular)
    
    _, contour, _ = cv2.findContours(circular.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    for i in contour:
        (x, y, w, h) = cv2.boundingRect(i)
        
        if ((cy + cy*0.25) > (y + h)) and ((circumference*0.25) > i.shape[0]):
            count += 1
            
    return count

In [6]:
#Main Function
cap = cv2.VideoCapture(0)
n_frame = 0

while True:
    
    
    
    _, frame = cap.read()
    
    frame = cv2.flip(frame, 1)
    
    frame1 = frame.copy()
    
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]
    
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7,7), 0)
    
    if n_frame < 60:
        running_bg(gray)
        if n_frame <= 59:
            cv2.putText(frame1, 'Wait', (400,400), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 7)
            cv2.imshow('main_frame', frame1)
    
    else:
        hand = segmenting(gray)
        if hand is not None:
            segment, thresh = hand
            
            cv2.drawContours(frame1, [segment + (roi_right, roi_top)], -1, (255,0,0), 1)
            
            count = fingers(segment, thresh)
            
            cv2.putText(frame1, str(count), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
            cv2.imshow("Thesholded", thresh)
                        
    cv2.rectangle(frame1, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)
        
    n_frame +=1
        
    cv2.imshow('main_frame', frame1)
        
    k = cv2.waitKey(20)
    if k==27:
        break
            
cap.release()
cv2.destroyAllWindows()